In [2]:
import pandas as pd

# Carga de datos

In [3]:
# Players DataFrame
players_df = pd.read_csv('players_stats.csv')
players_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column

# Teams Abreviation DataFrame
teams_abreviation_df = pd.read_csv('team_names.csv')
teams_abreviation_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column

# Teams DataFrame
teams_df = pd.read_csv('team_stats.csv')
teams_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
teams_df.insert(1, "Key",teams_abreviation_df['Key'], False) 

# Rookie DataFrame
rookies_df = pd.read_csv('rookie_stats.csv')
rookies_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column


# Limpieza de datos

## Borrado de columnas que no vamos a usar

In [4]:
players_df = players_df.drop('eFG%',1)
teams_df = teams_df.drop(['Div', 'MOV/A', 'ORtg/A', 'DRtg/A', 'NRtg/A'],1)
rookies_df = rookies_df.drop(['Yrs','MP.1', 'PTS.1', 'TRB.1', 'AST.1'], 1)

## Borrado de filas con valores nulos

In [5]:
players_df = players_df.dropna()
teams_df = teams_df.dropna()
rookies_df = rookies_df.dropna()

## Elminación de los jugadores que no tiene un equipo válido
El equipo de cada jugador debe estar en el dataframe de equipos identicado con el acrónimo correspondiente. Aunque, hay un caso excepcional, cuando un jugador ha estado en varios equipos durante la temporada, se guardan las estadistícas por separado y en total. En este último caso el equipo se identifica con TOT.

In [6]:
for i, row in players_df.iterrows():
    if row['Tm'] == 'TOT':
        continue
    
    if row['Tm'] not in str(teams_df['Key']):
        players_df = players_df.drop(index=i)

# Creación de la base de datos

## Creación del dataframe de la tabla central

In [7]:
central_df = pd.DataFrame(columns=['id_player', 'id_team', 'Player', 'Team','Rookie'])

central_df['id_player'] = players_df['id']
central_df['Player'] = players_df['Player']

for i, row in central_df.iterrows():
    # id team
    if players_df.loc[i]['Tm'] == 'TOT':
        row['id_team'] = 30
    else:
        row['id_team'] = teams_df.loc[list(teams_df['Key']).index(players_df.loc[i]['Tm'])]['id']


    # team name
    if row['id_team'] == 30:
        row['Team']='Total'
    else:
        row['Team'] = teams_df.loc[list(teams_df['id']).index(row['id_team'])]['Team']


    # is rookie
    if row['Player'] in str(rookies_df['Player']):
        row['Rookie'] = 1
    else:
        row['Rookie'] = 0

    central_df.loc[i] = row

,id_player,id_team,Player,Team,Rookie
0,0,9,Álex Abrines,Oklahoma City Thunder,0
2,2,26,Jaylen Adams,Atlanta Hawks,1
3,3,9,Steven Adams,Oklahoma City Thunder,0
4,4,15,Bam Adebayo,Miami Heat,0
5,5,30,Deng Adel,Total,1
...,...,...,...,...,...
721,721,24,Delon Wright,Memphis Grizzlies,0
722,722,8,Guerschon Yabusele,Boston Celtics,0
724,724,10,Thaddeus Young,Indiana Pacers,0
725,725,26,Trae Young,Atlanta Hawks,1
